In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os, sys

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
from math import log

%matplotlib inline

In [2]:
data  = pd.read_csv("../../data/train.csv")

In [3]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
data.shape

(1460, 81)

In [5]:
x = data.loc[:, data.columns != "SalePrice"]
y = data["SalePrice"]
x_train, x_test, y_train, y_test = train_test_split(x, y)
model = DummyRegressor().fit(x_train, y_train)
y_pred = model.predict(x_test)
y_test = [log(i) for i in y_test]
y_pred = [log(i) for i in y_pred]
metrics.mean_squared_error(y_test, y_pred, squared = False)

0.39802478792022056

In [6]:
ints = data.dtypes == 'int64'
floats = data.dtypes == 'float64'
numbers = floats | ints
numbers = numbers.loc[numbers].index

In [7]:
clean_data = data[data.columns.intersection(numbers)]
na_cols = clean_data.isna().any() == False
na_cols = na_cols.loc[na_cols].index
clean_data = clean_data[clean_data.columns.intersection(na_cols)]

In [8]:
def grade_model(model, data):
    x = data.iloc[:,:-1]
    y = data.iloc[:,data.shape[1] - 1]
    x_train, x_test, y_train, y_test = train_test_split(x, y)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    y_test = [log(i) for i in y_test]
    y_pred = [log(abs(i)) for i in y_pred]
    RMSE = metrics.mean_squared_error(y_test, y_pred)
    return RMSE


In [9]:
"""
x = clean_data.loc[:, clean_data.columns != "SalePrice"]
y = clean_data["SalePrice"]
x_train, x_test, prices_train, prices_test = train_test_split(x, y)

model = DecisionTreeRegressor(max_depth=40).fit(x_train, prices_train)
y_pred = model.predict(x_test)
metrics.mean_squared_error(prices_test, y_pred, squared = False)

model = LinearRegression().fit(x_train, prices_train)
y_pred = model.predict(x_test)
metrics.mean_squared_error(prices_test, y_pred, squared = False)

model = MLPRegressor(max_iter=1000).fit(x_train, prices_train)
y_pred = model.predict(x_test)
prices_test = [log(i) for i in prices_test]
y_pred = [log(i) for i in y_pred]
metrics.mean_squared_error(prices_test, y_pred, squared = False)
"""
None

In [11]:
model = DecisionTreeRegressor(max_depth=50)
a = grade_model(model, clean_data)
model = LinearRegression()
b = grade_model(model, clean_data)
model = RandomForestRegressor(max_depth=40 ,n_estimators = 50)
c = grade_model(model, clean_data)
print(a,b,c)

0.04078717938093267 0.04397031305679678 0.017972823320288763


In [12]:
x = clean_data.loc[:, clean_data.columns != "SalePrice"]
y = clean_data["SalePrice"]
model = RandomForestRegressor(max_depth=40 ,n_estimators = 50)
model.fit(x, y)
y_pred = model.predict(x)
y = [log(i) for i in y]
y_pred = [log(i) for i in y_pred]
RMSE = metrics.mean_squared_error(y, y_pred)
RMSE

0.003936531989595489

In [13]:
test_data = pd.read_csv("../../data/test.csv")

In [14]:
clean_test_data = test_data[test_data.columns.intersection(numbers)]
na_cols = clean_test_data.isna().any() == False
na_cols = na_cols.loc[na_cols].index
clean_test_data = clean_test_data[clean_test_data.columns.intersection(na_cols)]
clean_test_data

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,...,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1461,20,11622,5,6,1961,1961,896,0,0,...,0,140,0,0,0,120,0,0,6,2010
1,1462,20,14267,6,6,1958,1958,1329,0,0,...,0,393,36,0,0,0,0,12500,6,2010
2,1463,60,13830,5,5,1997,1998,928,701,0,...,1,212,34,0,0,0,0,0,3,2010
3,1464,60,9978,6,6,1998,1998,926,678,0,...,1,360,36,0,0,0,0,0,6,2010
4,1465,120,5005,8,5,1992,1992,1280,0,0,...,0,0,82,0,0,144,0,0,1,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,1936,4,7,1970,1970,546,546,0,...,0,0,0,0,0,0,0,0,6,2006
1455,2916,160,1894,4,5,1970,1970,546,546,0,...,0,0,24,0,0,0,0,0,4,2006
1456,2917,20,20000,5,7,1960,1996,1224,0,0,...,1,474,0,0,0,0,0,0,9,2006
1457,2918,85,10441,5,5,1992,1992,970,0,0,...,0,80,32,0,0,0,0,700,7,2006


In [15]:
x = data[data.columns.intersection(clean_test_data.columns)]
y = data["SalePrice"]
model = RandomForestRegressor(max_depth=40 ,n_estimators = 50)
model.fit(x, y)
y_pred = model.predict(x)
y = [log(i) for i in y]
y_pred = [log(i) for i in y_pred]
RMSE = metrics.mean_squared_error(y, y_pred)
RMSE

0.004179899143042629

In [ ]:
preds = model.predict(clean_test_data)
preds = pd.DataFrame({'ID' : test_data["Id"], "SalePrice" : preds}, columns=["ID", "SalePrice"])
preds.to_csv("predictions", index = False)